In [1]:
%pip install datasets evaluate
%pip install transformers[torch]
%pip install accelerate -U

# Please restart the runtime after installing packages

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 1. Data preparation

In [3]:
from datasets import load_dataset

imdb = load_dataset("imdb")

id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)


tokenized_imdb = imdb.map(preprocess_function, batched=True)

In [5]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 2. Model

In [6]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased",
                                                           num_labels=2,
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

# Training

In [8]:
import numpy as np
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [9]:
from transformers import TrainingArguments, Trainer


training_args = TrainingArguments(
    output_dir="out_dir",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/aivn12s1/.local/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/home/aivn12s1/.local/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /home/aivn12s1/miniconda3/envs/thuan_torch did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/home/aivn12s1/.local/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/local/cuda-11.8/lib64')}
  warn(msg)
/home/aivn12s1/.local/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /local/cuda-11.8/lib64: did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/home/aivn12s1/.local/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/org/gnome/Terminal/screen/de0e7c77_b63c_42f0_854c_0b1e5dc861c5')}
  warn(msg)
/home/aivn12s1/.local/lib/python3.10/site-packages/bit

  0%|          | 0/15630 [00:00<?, ?it/s]

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.3195, 'learning_rate': 1.9360204734484968e-05, 'epoch': 0.32}
{'loss': 0.2575, 'learning_rate': 1.872040946896993e-05, 'epoch': 0.64}
{'loss': 0.2321, 'learning_rate': 1.8080614203454897e-05, 'epoch': 0.96}


...

  0%|          | 0/1563 [00:00<?, ?it/s]

{'eval_loss': 0.516925573348999, 'eval_accuracy': 0.93056, 'eval_runtime': 230.2854, 'eval_samples_per_second': 108.561, 'eval_steps_per_second': 6.787, 'epoch': 9.0}
{'loss': 0.0056, 'learning_rate': 1.4459373000639796e-06, 'epoch': 9.28}
{'loss': 0.0023, 'learning_rate': 8.061420345489445e-07, 'epoch': 9.6}
{'loss': 0.0018, 'learning_rate': 1.6634676903390917e-07, 'epoch': 9.92}


  0%|          | 0/1563 [00:00<?, ?it/s]

{'eval_loss': 0.5505360960960388, 'eval_accuracy': 0.93156, 'eval_runtime': 230.2891, 'eval_samples_per_second': 108.559, 'eval_steps_per_second': 6.787, 'epoch': 10.0}
{'train_runtime': 9364.6981, 'train_samples_per_second': 26.696, 'train_steps_per_second': 1.669, 'train_loss': 0.06713474385263976, 'epoch': 10.0}


TrainOutput(global_step=15630, training_loss=0.06713474385263976, metrics={'train_runtime': 9364.6981, 'train_samples_per_second': 26.696, 'train_steps_per_second': 1.669, 'train_loss': 0.06713474385263976, 'epoch': 10.0})